In [1]:
# pip install --upgrade "openai>=1.4.0" requests

import requests, tempfile
from openai import OpenAI

client = OpenAI()                   # 키는 환경 변수에서 자동 로드

QUESTIONS = [
    "Aaronson 은 유죄인가요?",
    "그가 테이블에 어떤 메시지를 썼나요?",
    "Julia 는 누구인가요?",
]

# 1) Gist 원문을 임시 파일에 저장
URL = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw/"
txt = requests.get(URL, timeout=30).text
tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".txt")
tmp.write(txt.encode("utf-8")); tmp.close()

# 2) 파일 업로드 (assistants 용도)
file = client.files.create(file=open(tmp.name, "rb"), purpose="assistants")

# 3) 벡터스토어 생성 & 파일 등록  (beta 없이)
vs = client.vector_stores.create(name="aaronson_case")
client.vector_stores.files.create_and_poll(
    vector_store_id=vs.id,
    file_id=file.id
)

# 4) Assistant 생성 (File Search 도구 연결)
assistant = client.beta.assistants.create(
    name="RAG-Lite",
    instructions="근거 문장을 인용해 한국어로 간단히 답하세요.",
    model="gpt-4o-mini",
    tools=[{"type": "file_search"}],
    tool_resources={"file_search": {"vector_store_ids": [vs.id]}},
)

# 5) Thread 생성 + 질문 등록
thread = client.beta.threads.create()
for q in QUESTIONS:
    client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=q
    )

# 6) 실행 & 완료 대기
run = client.beta.threads.runs.create_and_poll(
    thread_id=thread.id,
    assistant_id=assistant.id
)

# 7) 답변 출력
for msg in reversed(client.beta.threads.messages.list(thread_id=thread.id).data):
    if msg.role == "assistant":
        print(msg.content[0].text.value)


C:\Users\ppjjh\AppData\Local\Temp\ipykernel_37648\637473913.py:40: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  thread = client.beta.threads.create()
C:\Users\ppjjh\AppData\Local\Temp\ipykernel_37648\637473913.py:42: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  client.beta.threads.messages.create(
C:\Users\ppjjh\AppData\Local\Temp\ipykernel_37648\637473913.py:49: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  run = client.beta.threads.runs.create_and_poll(
C:\Users\ppjjh\AppData\Local\Temp\ipykernel_37648\637473913.py:55: DeprecationWarning: The Assistants API is deprecated in favor of the Responses API
  for msg in reversed(client.beta.threads.messages.list(thread_id=thread.id).data):


1. **Aaronson의 유죄 여부**: Aaronson은 유죄로 판명되었습니다. "...Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with."【6:17†tmpratbbxv4.txt】.

2. **Aaronson이 쓴 메시지**: Aaronson은 "자유는 노예다"라는 메시지를 썼습니다. 이것은 그가 쓴 문구 중 하나로 "FREEDOM IS SLAVERY"라고 크게 썼습니다【6:17†tmpratbbxv4.txt】.

3. **Julia에 대한 정보**: Julia는 Winston의 사랑하는 사람입니다. 그녀는 Winston과의 관계에서의 배신과 그로 인해 변화한 감정에 대해 이야기합니다【6:12†tmpratbbxv4.txt】.


In [4]:
import openai, sys
print(openai.__version__)   # 예) '1.2.3' 처럼 출력되면 구버전


1.93.0
